In [1]:
import MySQLdb

In [2]:
import MySQLdb

class DbStreamer:

    def __init__(self, host, user, password, database):
        self.conn = MySQLdb.Connection(host=host,
                                       user=user,
                                       passwd=password,
                                       db=database,
                                       charset="utf8",
                                       use_unicode=True)
        return

    def get_connection(self):
        return self.conn

    def close_connection(self):
        self.conn.commit()
        self.conn.close()
        return

    def insert_user_details_sanitized(self, uid, uname):
        sql = "INSERT INTO user (userId, userName) VALUES (%s, %s)"
        val = (uid, uname)

        _cursor = self.conn.cursor()
        _cursor.execute(sql, val)
        return

    def insert_user_details_unsanitized(self, uid, uname):
        sql = "INSERT INTO user (userId, userName) VALUES ('%s', '%s')" % (uid, uname)

        _cursor = self.conn.cursor()
        _cursor.execute(sql)
        return
    
    def get_tables(self):
        sql = "SHOW TABLES;"
        
        _cursor = self.conn.cursor()
        _cursor.execute(sql)
        data = _cursor.fetchall()
        
        print(data)
        return

    def get_all_users(self):
        try:
            sql = "SELECT * FROM user"

            _cursor = self.conn.cursor()
            _cursor.execute(sql)

            row_headers = [x[0] for x in _cursor.description]
            data = _cursor.fetchall()

            json_data = []
            for result in data:
                json_data.append(dict(zip(row_headers, result)))
            print(json_data, end="\n\n")
        except Exception as ex:
            print(ex)
        return

### Initiate the connection

In [7]:
db_streamer = DbStreamer("localhost", username, password, "test")

### Get tables and all entries in user table

In [8]:
db_streamer.get_tables()
db_streamer.get_all_users()

(('user',),)
[]



### Sanitized input addition

In [9]:
db_streamer.insert_user_details_sanitized(1, "Alex A'); DROP TABLE user;")
db_streamer.get_tables()
db_streamer.get_all_users()

(('user',),)
[{'userId': 1, 'userName': "Alex A'); DROP TABLE user;"}]



### Unsanitized input

In [10]:
db_streamer.insert_user_details_unsanitized(2, "Bob S'); DROP TABLE user;")

In [12]:
db_streamer.get_all_users()

(1146, "Table 'test.user' doesn't exist")


In [13]:
db_streamer.get_tables()

()


### References:

[1] https://bobby-tables.com/python

[2] https://xkcd.com/327/